In [1]:
# Cargamos las librerias necesarias
import pandas as pd
from pathlib import Path

### Generar dataframe PRECIOS ELECTRICIDAD HORARIOS a partir de .txt de OMIE

## Definición directorios y archivos

In [2]:
# Directorio donde está el notebook
BASE_DIR = Path.cwd()
directorio = BASE_DIR / "Precios HORARIOS_Raw"

extension=".txt"
nombres_archivos = ["marginal_2019_TOTAL", "marginal_2020_TOTAL", "marginal_2021_TOTAL", "marginal_2022_TOTAL", "marginal_2023_TOTAL"]

## Funciones que generan dataframe

In [6]:
#_____________________________________________________________________________________
# 01.PRECIOS MARGINALES: FUNCIÓN ESPECÍFICA PARA ARCHIVO descargado desde OMIE
#_____________________________________________________________________________________

# FUNCIÓN B01_2: Generar DataFrame total desde los parciales y guardar CSV

def generate_df_prices_total(directorio, dir_base, nombre_archivos, extension):
    #"""
    # Genara 
    # df: DataFrame con columnas ['Year', 'Month', 'Day', 'Time', 'Price_1']
    #A partir de txt de cada año no separados por comas y con líneas no váidas 
    #"""
    df_total = pd.DataFrame() # Inicializar df_total
    for archivo in nombre_archivos:
        df_total = pd.concat([df_total, generate_df_prices_partial(directorio, archivo, extension)], ignore_index=True)    
        
    df_total.drop(columns=['Price_2'], inplace=True)  # Eliminar columna Price_2
    
   

    df_total.to_csv(f"{dir_base}/Precios_TOTAL.csv", index=False, sep=";")

    return df_total

# FUNCIÓN B01_1: Generar DataFrame parcial desde TXT, limpiar y guardar CSV
def generate_df_prices_partial(directorio, nombre_archivo, extension):
    
    data = []
    with open(f"{directorio}/{nombre_archivo}{extension}") as f:
        for line in f:
            cols = line.strip().split(";")     # separar por ";"
            cols = [c if c != "" else "0" for c in cols]  # convertir vacíos en 0
            data.append(cols)

    # Convertir a DataFrame rellenando con 0 donde falte
    df = pd.DataFrame(data).fillna(0)
    # Guardar CSV crudo (en ambas carpetas)
    df.to_csv(f"{directorio}/{nombre_archivo}_raw.csv", index=False, sep=";")

    # Damos nombre a cabeceras y Exploramos
    df.columns = ["Year", "Month", "Day", "Time", "Price_1", "Price_2", "NULL"]
    
    
    # Convertir Year a numérico, forzando errores a NaN
    df["Year"] = pd.to_numeric(df["Year"], errors="coerce")
    df.dropna(subset=["Year"], inplace=True)
    df.drop(columns=["NULL"], inplace=True)  # Eliminar columna NULL
    
    # Convertir a int
    df["Year"] = df["Year"].astype(int)
    df = df.astype({"Year": int, "Month": int, "Day": int, "Time": int, "Price_1": float, "Price_2": float})
    
    # explora_df(df,title_1=df.iloc[1,0])    
    
    # Guardar CSV limpio (en ambas carpetas)
    # df.to_csv(nombre_archivo + ".csv", index=False)  
    # df.to_csv(f"{directorio}/{nombre_archivo}.csv", index=False)
    return df 

## Llamada funciones

In [ ]:
df_precios = generate_df_prices_total(directorio,BASE_DIR, nombres_archivos, extension)

## Exploración dataframe generado

In [ ]:
def explora_df(df,title_1=""):
    print(f"------------EJEMPLO CABECERAS-{title_1}-------------\n--------------------------")
    print(df.head())
    print(f"------------INFO TIPO DATOS-{title_1}---------------\n--------------------------")
    df.info()
    print(f"------------ESTADÍSTICAS-{title_1} -----------------\n--------------------------")
    print(df.describe(include='all').transpose())
    
explora_df(df_precios)

------------EJEMPLO CABECERAS--------------
--------------------------
   Year  Month  Day  Time  Price_1
0  2019      1    1     1    66.88
1  2019      1    1     2    66.88
2  2019      1    1     3    66.00
3  2019      1    1     4    63.64
4  2019      1    1     5    58.85
------------INFO TIPO DATOS----------------
--------------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 43824 entries, 0 to 43823
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   Year     43824 non-null  int64  
 1   Month    43824 non-null  int64  
 2   Day      43824 non-null  int64  
 3   Time     43824 non-null  int64  
 4   Price_1  43824 non-null  float64
dtypes: float64(1), int64(4)
memory usage: 1.7 MB
------------ESTADÍSTICAS- -----------------
--------------------------
           count         mean        std     min      25%       50%       75%  \
Year     43824.0  2020.999452   1.414036  2019.0  2020.00  2021.000  202